### Use rio-viz in Jupyter Notebook

In [1]:
import time
import httpx

from folium import Map, TileLayer

from rio_viz.app import Client

In [2]:
# Create rio-viz Client (using server-thread to launch backgroud task)
client = Client("https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2573-1085/swissalti3d_2019_2573-1085_0.5_2056_5728.tif")

# Gives some time for the server to setup
time.sleep(1)

# Check that client is running
print("Client is alive: ", client.server.is_alive())

Client is alive:  True


In [3]:
# Fetch info
httpx.get(f"{client.endpoint}/info").json()

{'bounds': [7.090624928537461,
  45.916058441028206,
  7.1035698381384185,
  45.925093000254144],
 'band_metadata': [['1',
   {'STATISTICS_COVARIANCES': '10685.98787505646',
    'STATISTICS_EXCLUDEDVALUES': '-9999',
    'STATISTICS_MAXIMUM': '2015.0944824219',
    'STATISTICS_MEAN': '1754.471184271',
    'STATISTICS_MINIMUM': '1615.8128662109',
    'STATISTICS_SKIPFACTORX': '1',
    'STATISTICS_SKIPFACTORY': '1',
    'STATISTICS_STDDEV': '103.37305197708'}]],
 'band_descriptions': [['1', '']],
 'dtype': 'float32',
 'nodata_type': 'Nodata',
 'colorinterp': ['gray'],
 'overviews': [2, 4, 8],
 'height': 2000,
 'nodata_value': -9999.0,
 'count': 1,
 'driver': 'GTiff',
 'width': 2000}

In [4]:
r = httpx.get(
    f"{client.endpoint}/tilejson.json",
    params = {
        "rescale": "1600,2000",  # from the info endpoint
        "colormap_name": "terrain",
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
)
aod_layer.add_to(m)
m

/Users/vincentsarago/Dev/vincentsarago/rasterio/rasterio/io.py:140: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  return writer(mempath, 'w+', driver=driver, width=width,
ERROR 4: `/vsimem/c3a718d1-01e9-4503-8f86-519ae94e88c8/c3a718d1-01e9-4503-8f86-519ae94e88c8.tif' not recognized as a supported file format.
ERROR 4: `/vsimem/d141fe57-b93d-404c-805d-ff44812698e3/d141fe57-b93d-404c-805d-ff44812698e3.tif' not recognized as a supported file format.
ERROR 4: `/vsimem/ec85c789-d8a9-4bc6-ac09-e55b5498af6b/ec85c789-d8a9-4bc6-ac09-e55b5498af6b.tif' not recognized as a supported file format.
ERROR 4: `/vsimem/bd8ac208-f160-471d-be61-c9f42fe49fd7/bd8ac208-f160-471d-be61-c9f42fe49fd7.tif' not recognized as a supported file format.
